This example notebook shows how we can train an [image/digit classification](https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html)
model based on MNIST dataset, and store it as TileDB array. Firstly, let's import what we need.

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import tiledb
import json
import os
import glob

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn import tree

from pprint import pprint

from tiledb.ml.models.sklearn import SklearnTileDB

Then load our data, split in train and test and perform basic scaling by employing a standard scaler.

In [2]:
train_samples = 5000

# Load data from https://www.openml.org/d/554
print('Data fetching...')
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000)

print('Data scaling...')
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Data fetching...
Data scaling...


We move on by declaring a simple Logistic Regression classifier, train it and print the accuracy score.

In [3]:
clf = LogisticRegression(
    C=50. / train_samples, penalty='l1', solver='saga', tol=0.1
)

print('Model fit...')
clf.fit(X_train, y_train)

print('Model score...')
sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X_test, y_test)

print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

Model fit...
Model score...
Sparsity with L1 penalty: 78.12%
Test score with L1 penalty: 0.8309


We can now save the trained model as a TileDB array. For information about the structure of a dense
TileDB array in terms of files on disk please take a look [here](https://docs.tiledb.com/main/basic-concepts/data-format).
At the moment (will change in the future) we use pickle, which is one of the [most common scenarios for sklearn models](https://scikit-learn.org/stable/modules/model_persistence.html),
to serialize the whole model and save it as a [variable sized attribute](https://docs.tiledb.com/main/solutions/tiledb-embedded/api-usage/writing-arrays/var-length-attributes)
in a TileDB array.  We first declare a SklearnTileDB object (with the corresponding uri) and then save the model as a TileDB array.
Finally, we can save any kind of metadata (in any structure, i.e., list, tuple or dictionary) by passing a dictionary to the meta attribute.

In [4]:
tiledb_model_1 = SklearnTileDB(uri='tiledb-sklearn-mnist-1')

tiledb_model_1.save(model=clf,
                    meta={'Sparsity_with_L1_penalty': sparsity,
                          'score': score})

The above step will create a TileDB array in your working directory. Let's open our TileDB array model and check metadata. Metadata that are of type list, dict or tuple have been JSON
serialized while saving, i.e., we need json.loads to deserialize them.

In [9]:
# Check array directory
pprint(glob.glob('tiledb-sklearn-mnist-1/*'))

# Open in write mode in order to add metadata
print()
model_array_1 = tiledb.open('tiledb-sklearn-mnist-1')
for key, value in model_array_1.meta.items():
    if isinstance(value, bytes):
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value))

['tiledb-sklearn-mnist-1/__meta',
 'tiledb-sklearn-mnist-1/__lock.tdb',
 'tiledb-sklearn-mnist-1/__array_schema.tdb',
 'tiledb-sklearn-mnist-1/__1617026518252_1617026518252_90be26b060bd4818acd22bd13526beee_8',
 'tiledb-sklearn-mnist-1/__1617026518252_1617026518252_90be26b060bd4818acd22bd13526beee_8.ok']


Key: Sparsity_with_L1_penalty, Value: 78.125
Key: python_version, Value: 3.7.3
Key: score, Value: 0.8309
Key: sklearn_version, Value: 0.24.1


As we can see, in array's metadata we have by default information about the backend we used for training (sklearn),
sklearn version, python version and the extra metadata about epochs and training loss that we added.
We can load and check any of the aforementioned without having to load the entire model in memory.
Moreover, we can add any kind of extra information in model's metadata also by opening the TileDB array and adding new keys.

In [10]:
# Open the array in write mode
with tiledb.Array('tiledb-sklearn-mnist-1', "w") as A:
    # Keep all history
    A.meta['new_meta'] = json.dumps(['Any kind of info'])

# Check that everything is there
model_array_1 = tiledb.open('tiledb-sklearn-mnist-1')
for key, value in model_array_1.meta.items():
    if isinstance(value, bytes):
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value))

Key: Sparsity_with_L1_penalty, Value: 78.125
Key: new_meta, Value: ["Any kind of info"]
Key: python_version, Value: 3.7.3
Key: score, Value: 0.8309
Key: sklearn_version, Value: 0.24.1


Moving on, we can load the trained models for evaluation or retraining, as usual with Sklearn models.
What is really nice with saving models as TileDB array, is native versioning based on fragments as described
[here](https://docs.tiledb.com/main/basic-concepts/data-format#immutable-fragments). We can load a model, retrain it
with new data and update the already existing TileDB model array with new model parameters and metadata. All information, old
and new will be there and accessible. This is extremely useful when you retrain with new data or trying different architectures
for the same problem, and you want to keep track of all your experiments without having to store different model instances. In our case,
let's continue training tiledb-sklearn-mnist-1 with test data (just for simplicity). After training is done, we can save the model
again with update=True. You will notice the extra directories and files (fragments) added to tiledb-sklearn-mnist-1 TileDB array directory,
which keep all versions of the model.

In [13]:
loaded_clf = tiledb_model_1.load()

# Sparsity and score should be the same as in the previous step.
print('Model score...')
sparsity = np.mean(loaded_clf.coef_ == 0) * 100
score = loaded_clf.score(X_test, y_test)

print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)


# We retrain with test data just for the sake of simplicity.
print('Model fit...')
loaded_clf.fit(X_test, y_test)

print('Model score...')
sparsity = np.mean(loaded_clf.coef_ == 0) * 100
score = loaded_clf.score(X_test, y_test)

print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

tiledb_model_1.save(model=loaded_clf,
                    update=True,
                    meta={'Sparsity_with_L1_penalty': sparsity,
                          'score': score})

# Check array directory
print()
pprint(glob.glob('tiledb-sklearn-mnist-1/*'))


# tiledb.array_fragments() requires TileDB-Py version > 0.8.5
fragments_info = tiledb.array_fragments('tiledb-sklearn-mnist-1')

print()
print("====== FRAGMENTS  INFO ======")
print("array uri: {}".format(fragments_info.array_uri))
print("number of fragments: {}".format(len(fragments_info)))

for fragment_num, fragment in enumerate(fragments_info, start=1):
    print()
    print("===== FRAGMENT NUMBER {} =====".format(fragment.num))
    print("fragment uri: {}".format(fragment.uri))
    print("is dense: {}".format(fragment.dense))
    print("is sparse: {}".format(fragment.sparse))
    print("cell num: {}".format(fragment.cell_num))
    print("has consolidated metadata: {}".format(fragment.has_consolidated_metadata))
    print("non empty domain: {}".format(fragment.non_empty_domain))
    print("timestamp range: {}".format(fragment.timestamp_range))
    print("number of fragments to vacuum: {}".format(fragment.to_vacuum_num))
    print("uri of fragments to vacuum: {}".format(fragment.to_vacuum_uri))
    print(
        "number of unconsolidated metadata: {}".format(
            fragment.unconsolidated_metadata_num
        )
    )
    print("version: {}".format(fragment.version))


Model score...
Sparsity with L1 penalty: 45.03%
Test score with L1 penalty: 0.7293
Model fit...
Model score...
Sparsity with L1 penalty: 44.76%
Test score with L1 penalty: 0.7305

['tiledb-sklearn-mnist-1/__1617026714948_1617026714948_5d812d9258304da89d625c3b1a0d37f3_8',
 'tiledb-sklearn-mnist-1/__1617026714948_1617026714948_5d812d9258304da89d625c3b1a0d37f3_8.ok',
 'tiledb-sklearn-mnist-1/__meta',
 'tiledb-sklearn-mnist-1/__lock.tdb',
 'tiledb-sklearn-mnist-1/__array_schema.tdb',
 'tiledb-sklearn-mnist-1/__1617026796481_1617026796481_28a4ea23102d40099ceb1772626513a4_8.ok',
 'tiledb-sklearn-mnist-1/__1617026796481_1617026796481_28a4ea23102d40099ceb1772626513a4_8',
 'tiledb-sklearn-mnist-1/__1617026632600_1617026632600_43ccd847df854db3bb5998750aef7143_8',
 'tiledb-sklearn-mnist-1/__1617026518252_1617026518252_90be26b060bd4818acd22bd13526beee_8',
 'tiledb-sklearn-mnist-1/__1617026632600_1617026632600_43ccd847df854db3bb5998750aef7143_8.ok',
 'tiledb-sklearn-mnist-1/__1617026518252_16170265

Finally, a very interesting and useful, for machine learning models, TileDB feature that is described
[here](https://docs.tiledb.com/main/basic-concepts/data-format#groups) and [here](https://docs.tiledb.com/main/solutions/tiledb-embedded/api-usage/object-management#creating-tiledb-groups)
are groups. Assuming we want to solve the MNIST problem, and we want to try several architectures. We can save each architecture
as a separate TileDB array with native versioning each time it is re-trained, and then organise all models that solve the same problem (MNIST)
as a TileDB array group with any kind of hierarchy. Let's firstly define a new model architecture, then train a model and save
it as a new TileDB array.

In [14]:
# We declare a Decision Tree classifier
clf = tree.DecisionTreeClassifier()

# Fit the model
print('Fit...')
clf.fit(X_train, y_train)

# Evaluate
score = clf.score(X_test, y_test)
print("Test score: %.4f" % score)

# Declare a SklearnTileDB object
tiledb_model_2 = SklearnTileDB(uri='tiledb-sklearn-mnist-2')

# Save model as a TileDB array
tiledb_model_2.save(model=clf,
                    meta={'score': score})

Fit...
Test score: 0.7729


Now we can create a TileDB group and organise (in hierarchies, e.g., sophisticated vs less sophisticated) all our
MNIST models as follows.

In [15]:
tiledb.group_create('MNIST_Group')
os.system('mv tiledb-sklearn-mnist-1 MNIST_Group/')
os.system('mv tiledb-sklearn-mnist-2 MNIST_Group/')

0

Any time we can check and query all the available models, including their metadata, for a specific problem like MNIST.

In [16]:
tiledb.ls('MNIST_Group', lambda obj_path, obj_type: print(obj_path, obj_type))

file:///Users/george/PycharmProjects/TileDB-ML/example_notebooks/models/MNIST_Group/tiledb-sklearn-mnist-1 array
file:///Users/george/PycharmProjects/TileDB-ML/example_notebooks/models/MNIST_Group/tiledb-sklearn-mnist-2 array
